In [1]:
import trimesh
import numpy as np
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume
from irregular_object_packing.mesh.utils import print_mesh_info
from irregular_object_packing.packing import initialize, nlc_optimisation as nlc, chordal_axis_transform as cat
from irregular_object_packing.packing.growth_based_optimisation import SimSettings, Optimizer
from importlib import reload

mesh_volume = 0.3
container_volume = 5
coverage_rate = 0.3

DATA_FOLDER = "../data/mesh/"
loaded_mesh = trimesh.load_mesh(DATA_FOLDER + "RBC_normal.stl")
print_mesh_info(loaded_mesh, "loaded mesh")
# trimesh.Scene([loaded_mesh]).show()
# Scale the mesh to the desired volume
original_mesh = scale_and_center_mesh(loaded_mesh, mesh_volume)
print_mesh_info(original_mesh, "scaled mesh")

container = trimesh.primitives.Sphere(radius=1, center=[0, 0, 0])
# container = trimesh.primitives.Cylinder(radius=1, height=1)
print_mesh_info(container, "original container")

container = scale_to_volume(container, container_volume)
print_mesh_info(container, "scaled container")

Mesh info loaded mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, 
volume: 71.04896973139347, 
bounding box: [[35.089 37.963 35.089]
 [42.911 40.037 42.911]] 
center of mass: [39. 39. 39.]

Mesh info scaled mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, 
volume: 0.30000000000000027, 
bounding box: [[-0.6321 -0.1676 -0.6321]
 [ 0.6321  0.1676  0.6321]] 
center of mass: [0. 0. 0.]

Mesh info original container: <trimesh.primitives.Sphere>, 
volume: 4.1887902047863905, 
bounding box: [[-1. -1. -1.]
 [ 1.  1.  1.]] 
center of mass: [-0.  0.  0.]

Mesh info scaled container: <trimesh.primitives.Sphere>, 
volume: 5.000000000000001, 
bounding box: [[-1.0608 -1.0608 -1.0608]
 [ 1.0608  1.0608  1.0608]] 
center of mass: [0. 0. 0.]



In [2]:
import irregular_object_packing.packing.growth_based_optimisation as gbo
reload(gbo)


settings = gbo.SimSettings(sample_rate=30, max_angle=1 / 12 * np.pi, max_t=None, initial_scale=0.1, itn_max=1, n_scaling_steps=2, coverage_rate=0.3,)

optimizer = gbo.Optimizer([original_mesh], container, settings)

optimizer.run()


! collision found for 3 objectts with total of 3 contacts


scaling_barrier:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s['obj_id', {'value': 0}]]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s['obj_id', {'value': 0}]]

! collision found for 3 objectts with total of 3 contacts
Collision: None
Optimisation complete. scales: [0.9999999999999999, 0.9999999999999907, 1.0, 0.9999999999999999, 0.9999999999999999]


In [10]:
import irregular_object_packing.packing.plots as plots
reload(plots)
object_id = 1
first = 0
second = 1

cat_cell_mesh_0 = cat.cat_mesh_from_data(optimizer.cat_log[first], object_id)
# cat_cell_mesh_1 = cat.cat_mesh_from_data(optimizer.cat_log[second], object_id)


plots.plot_step_comparison(
    original_mesh,
    [optimizer.tf_log[first][object_id], optimizer.tf_log[second][object_id]],
    cat_cell_mesh_0
    # cat_cell_mesh_1,
)

# %%
# print(optimizer.tf_log[first][object_id])
# print(optimizer.tf_log[second][object_id])


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [4]:
optimizer.cat_log[second][object_id]

KeyError: 2

In [ ]:

# Plot of the final packing
import pyvista as pv
# from irregular_object_packing.packing.plots import create_plot, plot_stat
import irregular_object_packing.packing.plots as plots

optimizer.transform_data

# plots.plot_state(optimizer)
